### Pregunta 1

¿Cuánto es el máximo que podemos llegar a pagar en un día por los servidores?

In [29]:

def servers_price(num_nodes: int) -> int:
    normal_price: int = 130
    discounted_price: int = 110
    
    total = []
    
    for num in range(1,(num_nodes+1)):
        if num % 3 == 0:
            total.append(discounted_price)
        else:
            total.append(normal_price)
    
    return sum(total)

number_of_analysts = 14

print(servers_price(num_nodes=number_of_analysts), "€ pagando un nodo por analista al día.")

1740 € pagando un nodo por analista al día.


### Pregunta 2

Pinta un histograma de la distribución de los costes diarios realizando simulaciones de Monte Carlo.

¿Cuál es el coste medio por día que esperamos tener que pagar por los servidores, si siempre pagamos todos los servidores que hacen falta en un día dado?